In [23]:
import pandas as pd
import json
import os
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def solve_vrp_final_fixed_split():
    print("🚀 Memulai Optimasi FINAL: Split Node + Fixed 3 Truk + Multi-Trip...")
    
    # ==========================================
    # 1. KONFIGURASI
    # ==========================================
    BASE_DIR = os.getcwd()
    if os.path.basename(BASE_DIR) == 'notebook':
        BASE_DIR = os.path.dirname(BASE_DIR)
    DATA_DIR = os.path.join(BASE_DIR, 'dataset')
    RES_DIR = os.path.join(BASE_DIR, 'results')
    os.makedirs(RES_DIR, exist_ok=True)
    
    CSV_FILE = os.path.join(DATA_DIR, 'final_dataset_2echelon.csv')
    JSON_FILE = os.path.join(DATA_DIR, 'distance_matrix_osrm.json')
    REPORT_FILE = os.path.join(RES_DIR, 'reports.txt')
    
    # --- PARAMETER UTAMA ---
    FIXED_TRUCK_COUNT = 6
    TRUCK_CAPACITY = 6.0    # Ton per Rit
    MAX_SHIFT_HOURS = 8     # Jam kerja normal
    
    # Parameter Biaya & Waktu
    COST_PER_KM = 15000
    TRUCK_SPEED_KMH = 30
    LOADING_TIME_MINS = 15
    UNLOADING_TIME_MINS = 30
    
    # ==========================================
    # 2. DATA LOAD & SPLIT LOGIC
    # ==========================================
    if not os.path.exists(CSV_FILE):
        print("❌ File CSV tidak ditemukan.")
        return

    df_orig = pd.read_csv(CSV_FILE)
    
    with open(JSON_FILE, 'r') as f:
        data_json = json.load(f)
        full_matrix = data_json['distance_matrix']
        if 'node_ids' in data_json:
            id_to_idx = {nid: i for i, nid in enumerate(data_json['node_ids'])}
        else:
            id_to_idx = {nid: i for i, nid in enumerate(df_orig['Node_ID'].tolist())}

    tpa_row = df_orig[df_orig['Type'] == 'TPA']
    if len(tpa_row) == 0: return
    tpa_id = tpa_row.iloc[0]['Node_ID']
    
    all_raw_trips = []
    
    # ==========================================
    # 3. PHASE 1: GENERATE RIT (DENGAN SPLIT NODE)
    # ==========================================
    print("🔄 Phase 1: Memecah TPS besar & menghitung Ritase...")
    
    kecamatan_ids = df_orig[df_orig['Type'] == 'TPS']['Kecamatan_ID'].unique()
    
    for kec_id in sorted(kecamatan_ids):
        pool_rows = df_orig[(df_orig['Kecamatan_ID'] == kec_id) & (df_orig['Type'] == 'POOL')]
        tps_rows_orig = df_orig[(df_orig['Kecamatan_ID'] == kec_id) & (df_orig['Type'] == 'TPS')]
        
        if len(pool_rows) == 0 or len(tps_rows_orig) == 0: continue
        pool_id = pool_rows.iloc[0]['Node_ID']
        
        # --- SPLIT NODE LOGIC ---
        # Memecah TPS > 6 Ton menjadi beberapa node virtual
        solver_nodes = []
        
        # 1. Pool
        solver_nodes.append({'id': pool_id, 'demand': 0, 'idx': id_to_idx[pool_id]})
        
        # 2. TPS (Split)
        for _, row in tps_rows_orig.iterrows():
            d = row['Demand_Ton']
            nid = row['Node_ID']
            # Pecah demand
            while d > 0.01: # Pakai toleransi kecil
                chunk = min(d, TRUCK_CAPACITY)
                solver_nodes.append({'id': nid, 'demand': chunk, 'idx': id_to_idx[nid]})
                d -= chunk
        
        # 3. TPA
        solver_nodes.append({'id': tpa_id, 'demand': 0, 'idx': id_to_idx[tpa_id]})
        
        # --- PREPARE SOLVER ---
        n_nodes = len(solver_nodes)
        sub_matrix = [[0]*n_nodes for _ in range(n_nodes)]
        demands = [n['demand'] for n in solver_nodes]
        
        for i in range(n_nodes):
            for j in range(n_nodes):
                u, v = solver_nodes[i]['idx'], solver_nodes[j]['idx']
                sub_matrix[i][j] = full_matrix[u][v]
        
        # Jumlah Rit Estimasi (Virtual Vehicles)
        # Kita set longgar agar solver bisa menemukan solusi optimal
        num_virtual_vehicles = len(solver_nodes) 
        
        manager = pywrapcp.RoutingIndexManager(n_nodes, num_virtual_vehicles, [0]*num_virtual_vehicles, [n_nodes-1]*num_virtual_vehicles)
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            return sub_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]
        
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        def demand_callback(from_index):
            return int(demands[manager.IndexToNode(from_index)] * 100)
            
        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [int(TRUCK_CAPACITY*100)]*num_virtual_vehicles, True, 'Capacity')

        # Penalty tinggi untuk node yang tidak terangkut
        for i in range(1, n_nodes - 1):
            routing.AddDisjunction([manager.NodeToIndex(i)], 10000000)

        search_params = pywrapcp.DefaultRoutingSearchParameters()
        search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        search_params.time_limit.seconds = 1

        solution = routing.SolveWithParameters(search_params)
        
        if solution:
            for v_id in range(num_virtual_vehicles):
                if not routing.IsVehicleUsed(solution, v_id): continue
                
                index = routing.Start(v_id)
                route_nodes = []
                route_dist = 0
                route_load = 0
                tps_visit_count = 0
                
                while not routing.IsEnd(index):
                    node_idx = manager.IndexToNode(index)
                    route_nodes.append(int(solver_nodes[node_idx]['id']))
                    
                    if 0 < node_idx < n_nodes-1:
                        route_load += demands[node_idx]
                        tps_visit_count += 1
                        
                    prev = index
                    index = solution.Value(routing.NextVar(index))
                    route_dist += routing.GetArcCostForVehicle(prev, index, v_id)
                
                # Add TPA
                route_nodes.append(int(solver_nodes[manager.IndexToNode(index)]['id']))
                
                if route_load > 0.01:
                    # Hitung Waktu Rit
                    t_travel = (route_dist / 1000) / TRUCK_SPEED_KMH
                    t_load = (tps_visit_count * LOADING_TIME_MINS) / 60
                    t_unload = UNLOADING_TIME_MINS / 60
                    total_time = t_travel + t_load + t_unload
                    
                    all_raw_trips.append({
                        "Kecamatan_ID": int(kec_id),
                        "Nodes": route_nodes,
                        "Load": round(route_load, 2),
                        "Dist_KM": round(route_dist / 1000, 2),
                        "Duration_Hours": round(total_time, 2),
                        "Cost": int((route_dist / 1000) * COST_PER_KM)
                    })

    # ==========================================
    # 4. PHASE 2: ASSIGN KE 3 TRUK FISIK (LOAD BALANCING)
    # ==========================================
    print(f"🔄 Phase 2: Mendistribusikan {len(all_raw_trips)} Rit ke {FIXED_TRUCK_COUNT} Truk Fisik per Kecamatan...")
    
    final_fleet = []
    
    for kec_id in sorted(kecamatan_ids):
        # Ambil rit kecamatan ini
        kec_trips = [t for t in all_raw_trips if t['Kecamatan_ID'] == kec_id]
        # Sort dari yang paling lama durasinya (Best Practice Bin Packing)
        kec_trips.sort(key=lambda x: x['Duration_Hours'], reverse=True)
        
        # Siapkan 3 Truk Kosong
        trucks = [{'trips': [], 'total_dur': 0} for _ in range(FIXED_TRUCK_COUNT)]
        
        for trip in kec_trips:
            # Cari truk dengan beban waktu terendah saat ini (Greedy Load Balancing)
            # Agar pekerjaan terbagi rata
            best_truck = min(trucks, key=lambda x: x['total_dur'])
            best_truck['trips'].append(trip)
            best_truck['total_dur'] += trip['Duration_Hours']
            
        # Format Output
        for i, t_data in enumerate(trucks):
            if not t_data['trips']: continue # Skip kalau truk nganggur (jarang terjadi)
            
            trips = t_data['trips']
            tot_load = sum(t['Load'] for t in trips)
            tot_dist = sum(t['Dist_KM'] for t in trips)
            tot_cost = sum(t['Cost'] for t in trips)
            tot_time = t_data['total_dur']
            
            # Status Overtime
            status = "AMAN" if tot_time <= MAX_SHIFT_HOURS else f"OVERTIME (+{tot_time-MAX_SHIFT_HOURS:.1f}h)"
            
            # Merge Nodes for Map
            merged_nodes = []
            rit_details = []
            for j, tr in enumerate(trips):
                if j==0: merged_nodes.extend(tr['Nodes'])
                else: merged_nodes.extend(tr['Nodes']) # Sambung
                
                rit_details.append({
                    "Rit_Ke": j+1, "Load": tr['Load'], "Dur": tr['Duration_Hours']
                })
            
            final_fleet.append({
                "Kecamatan_ID": int(kec_id),
                "Vehicle": f"Kec{kec_id}_TRUK_{i+1}",
                "Route_Nodes": merged_nodes,
                "Total_Load": round(tot_load, 2),
                "Total_Dist": round(tot_dist, 2),
                "Total_Time": round(tot_time, 2),
                "Status": status,
                "Rit_Count": len(trips),
                "Details": rit_details,
                "Cost": tot_cost
            })

    # ==========================================
    # 5. REPORTING
    # ==========================================
    total_gerobak = df_orig['Cost_Gerobak'].sum()
    total_truk_cost = sum(f['Cost'] for f in final_fleet)
    
    # Hitung Coverage
    total_demand_real = df_orig[df_orig['Type']=='TPS']['Demand_Ton'].sum()
    total_angkut = sum(f['Total_Load'] for f in final_fleet)
    
    report = [
        "="*80,
        f"       LAPORAN FINAL VRP (METODE: SPLIT NODE + FIXED {FIXED_TRUCK_COUNT} TRUCK)",
        "="*80,
        f"1. Total Sampah Asli  : {total_demand_real:,.2f} Ton",
        f"2. Total Terangkut    : {total_angkut:,.2f} Ton",
        f"   (Coverage: {total_angkut/total_demand_real*100:.1f}%) -> HARUS 100%",
        f"3. Total Ritase       : {len(all_raw_trips)} Trip",
        "-"*80,
        "ANALISIS BEBAN KERJA PER KECAMATAN:",
        f"{'Kec':<4} | {'Truk':<6} | {'Rit':<5} | {'Sampah':<10} | {'Jam Kerja (Avg)':<18} | {'Status'}"
    ]
    
    # Aggregasi Per Kecamatan
    kec_summary = {}
    for f in final_fleet:
        k = f['Kecamatan_ID']
        if k not in kec_summary: kec_summary[k] = {'truk':0, 'rit':0, 'load':0, 'time':0, 'ot':0}
        kec_summary[k]['truk'] += 1
        kec_summary[k]['rit'] += f['Rit_Count']
        kec_summary[k]['load'] += f['Total_Load']
        kec_summary[k]['time'] += f['Total_Time']
        if "OVERTIME" in f['Status']: kec_summary[k]['ot'] += 1
        
    for k in sorted(kec_summary.keys()):
        s = kec_summary[k]
        avg_time = s['time']/s['truk']
        stat = "✅ AMAN" if s['ot'] == 0 else f"⚠️ {s['ot']} TRUK LEMBUR"
        report.append(f"{k:<4} | {s['truk']:<6} | {s['rit']:<5} | {s['load']:<10.1f} | {avg_time:<18.2f} | {stat}")
        
    report.extend([
        "-"*80,
        f"BIAYA ECHELON 1 (GEROBAK) : Rp {total_gerobak:,.0f}",
        f"BIAYA ECHELON 2 (TRUK)    : Rp {total_truk_cost:,.0f}",
        f"💰 GRAND TOTAL            : Rp {total_gerobak + total_truk_cost:,.0f}",
        "="*80
    ])
    
    for l in report: print(l)
    
    with open(REPORT_FILE, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report))
    
    with open(os.path.join(RES_DIR, 'optimized_routes.json'), 'w', encoding='utf-8') as f:
        json.dump(final_fleet, f, indent=4)
        
    print(f"\n✅ Selesai. Data tersimpan di {RES_DIR}")

if __name__ == "__main__":
    solve_vrp_final_fixed_split()

🚀 Memulai Optimasi FINAL: Split Node + Fixed 3 Truk + Multi-Trip...
🔄 Phase 1: Memecah TPS besar & menghitung Ritase...
🔄 Phase 2: Mendistribusikan 211 Rit ke 6 Truk Fisik per Kecamatan...
       LAPORAN FINAL VRP (METODE: SPLIT NODE + FIXED 6 TRUCK)
1. Total Sampah Asli  : 1,185.60 Ton
2. Total Terangkut    : 1,185.60 Ton
   (Coverage: 100.0%) -> HARUS 100%
3. Total Ritase       : 211 Trip
--------------------------------------------------------------------------------
ANALISIS BEBAN KERJA PER KECAMATAN:
Kec  | Truk   | Rit   | Sampah     | Jam Kerja (Avg)    | Status
0    | 6      | 10    | 59.2       | 4.24               | ✅ AMAN
1    | 6      | 18    | 107.9      | 7.24               | ⚠️ 1 TRUK LEMBUR
2    | 6      | 7     | 39.3       | 2.70               | ✅ AMAN
3    | 6      | 7     | 39.7       | 2.65               | ✅ AMAN
4    | 2      | 2     | 6.9        | 2.79               | ✅ AMAN
5    | 6      | 11    | 63.0       | 4.30               | ✅ AMAN
6    | 6      | 7     | 